### Import required modules

In [1]:
import json
import sys
import urllib
import argparse
from urllib.parse import urlparse, urlencode, parse_qs
from urllib.request import  urlopen
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout

### Youtube Comment API url

In [2]:
YOUTUBE_COMMENT_URL = 'https://www.googleapis.com/youtube/v3/commentThreads'

### Class defined to fetch comments of provided youtube video

In [3]:
#define class
class YouTubeApi():
    
    #define dataframe to save comments fetched from url
    df = pd.DataFrame(columns=['AuthorName','Comment'])
    
    # function to fetch comments from youtube url
    def load_comments(self, mat):
        
        for item in mat["items"]:
            comment = item["snippet"]["topLevelComment"]
            author = comment["snippet"]["authorDisplayName"]
            text = comment["snippet"]["textDisplay"]
            self.df = self.df.append({'AuthorName' : author, 'Comment' : text} , ignore_index=True)
            if 'replies' in item.keys():
                for reply in item['replies']['comments']:
                    rauthor = reply['snippet']['authorDisplayName']
                    rtext = reply["snippet"]["textDisplay"]
                    self.df = self.df.append({'AuthorName' : rauthor, 'Comment' : rtext} , ignore_index=True)

    # provide maximum number of comments to fetch per page(maximum is 100), youtube video url and youtube api key
    # you can get youtube api key from Google Clouds API page
    def get_video_comment(self, maximum, videoUrl, key): 
    
        vid = str()
        matches = {}

        try:
            video_id = urlparse(str(videoUrl))
            q = parse_qs(video_id.query)
            vid = q["v"][0]

        except:
            print("Invalid YouTube URL")

        parms = {
                    'part': 'snippet,replies',
                    'maxResults': maximum,
                    'videoId': vid,
                    'textFormat': 'plainText',
                    'key': key
                }

        try:

            matches = self.openURL(YOUTUBE_COMMENT_URL, parms)
            i = 2
            mat = json.loads(matches)
            nextPageToken = mat.get("nextPageToken")
            print("\nPage : 1 - " + str(nextPageToken))
            print("------------------------------------------------------------------")
            self.load_comments(mat)

            while nextPageToken:
                parms.update({'pageToken': nextPageToken})
                matches = self.openURL(YOUTUBE_COMMENT_URL, parms)
                mat = json.loads(matches)
                nextPageToken = mat.get("nextPageToken")
                print("\nPage : " + str(i) + " - " + str(nextPageToken))
                print("------------------------------------------------------------------")

                self.load_comments(mat)

                i += 1
             
            self.df.to_csv('output.csv')
            
        except KeyboardInterrupt:
            print("User Aborted the Operation")

        except Exception as e:
            print(e)
            print(json.loads(matches))


    # function to read data from comments api url
    def openURL(self, url, parms):
            f = urlopen(url + '?' + urlencode(parms))
            data = f.read()
            f.close()
            matches = data.decode("utf-8")
            return matches


### Define visual elements to collect input

In [4]:
inputBoxVideoUrl = widgets.Text(
    placeholder='Provide Youtube video url',
    disabled=False,
    layout=Layout(width='100%'))

inputBoxAPIKey = widgets.Text(
    placeholder='Provide Youtube API Key',
    disabled=False,
    layout=Layout(width='100%'))

button = widgets.Button(
    description='Fetch Comments',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

out = widgets.Output()

def fetch_commnets():
    inpOne = inputBoxVideoUrl.value
    inpTwo = inputBoxAPIKey.value
    out.clear_output()
    if (inpOne != '' and inpTwo != ''):
        y = YouTubeApi()
        y.get_video_comment(100, inpOne, inpTwo)
    else:
        with out:
            print("Please provide value for both video urland API key")

def on_button_clicked(b):
    fetch_commnets()
            
button.on_click(on_button_clicked)

### Provide required values below and click on button
#### You can get the Youtube API Key from your google cloud platform account

In [5]:
display(inputBoxVideoUrl)
display(inputBoxAPIKey)
display(button)
display(out)

Text(value='', layout=Layout(width='100%'), placeholder='Provide Youtube video url')

Text(value='', layout=Layout(width='100%'), placeholder='Provide Youtube API Key')

Button(button_style='info', description='Fetch Comments', style=ButtonStyle(), tooltip='Click me')

Output()